In [ ]:
# Fetching Bitcoin open interest history from Coinalyze API
import os
import requests 
import pandas as pd

API_KEY = os.getenv("COINALYZE_API_KEY")
if not API_KEY:
    raise RuntimeError("COINALYZE_API_KEY not set in env")

SYMBOL   = "BTCUSDT_PERP.A"
INTERVAL = "daily"
START_TS = int(pd.to_datetime("2020-04-15").timestamp())
END_TS   = int(pd.to_datetime("2025-04-29").timestamp())

url = "https://api.coinalyze.net/v1/open-interest-history"
params = {
    "symbols":        SYMBOL,
    "interval":       INTERVAL,
    "from":           START_TS,
    "to":             END_TS,
    "convert_to_usd": "true",
    "api_key":        API_KEY
}
resp = requests.get(url, params=params)
resp.raise_for_status()  
data = resp.json()       

# Picking BTC open interest history
hist = data[0]["history"]

df_oi = (
    pd.DataFrame(hist)
      .rename(columns={
          "t": "timestamp",
          "o": "open_interest",
          "h": "high_oi",
          "l": "low_oi",
          "c": "close_oi"
      })
)
# convert UNIX seconds → datetime index
df_oi["timestamp"] = pd.to_datetime(df_oi["timestamp"], unit="s")
df_oi = df_oi.set_index("timestamp")

df_oi.head()


,open_interest,high_oi,low_oi,close_oi
timestamp,,,,
2021-03-19,1.878231e+09,1.929666e+09,1.834906e+09,1.877387e+09
2021-03-20,1.879213e+09,1.964769e+09,1.840307e+09,1.882912e+09
2021-03-21,1.859258e+09,1.956365e+09,1.795490e+09,1.860386e+09
2021-03-22,1.753924e+09,1.886354e+09,1.740609e+09,1.801493e+09
2021-03-23,1.810493e+09,1.928055e+09,1.769320e+09,1.865623e+09


In [5]:
df_oi.tail()


,open_interest,high_oi,low_oi,close_oi
timestamp,,,,
2025-04-25,7.835888e+09,8.237823e+09,7.734183e+09,8.133263e+09
2025-04-26,8.131747e+09,8.136491e+09,7.911268e+09,7.931770e+09
2025-04-27,7.858064e+09,7.946920e+09,7.813984e+09,7.882132e+09
2025-04-28,7.989089e+09,8.091449e+09,7.745728e+09,7.776752e+09
2025-04-29,7.752826e+09,7.840178e+09,7.741612e+09,7.745374e+09


In [6]:
df_oi.to_csv('../output/bitcoin_open_interest.csv')

In [ ]:
# Fetch funding rates history from Coinalyze API
import os
import requests 
import pandas as pd

API_KEY = os.getenv("COINALYZE_API_KEY")
if not API_KEY:
    raise RuntimeError("COINALYZE_API_KEY not set in env")

SYMBOL   = "BTCUSDT_PERP.A"
INTERVAL = "daily"
START_TS = int(pd.to_datetime("2020-04-15").timestamp())
END_TS   = int(pd.to_datetime("2025-04-29").timestamp())

url = "https://api.coinalyze.net/v1/funding-rate-history"
params = {
    "symbols":        SYMBOL,
    "interval":       INTERVAL,
    "from":           START_TS,
    "to":             END_TS,
    "api_key":        API_KEY
}
resp = requests.get(url, params=params)
resp.raise_for_status()  
data = resp.json()      

# Picking BTC funding rates history
hist = data[0]["history"]

df_fr = (
    pd.DataFrame(hist)
      .rename(columns={
          "t": "timestamp",
          "o": "open_rate",
          "h": "high_rate",
          "l": "low_rate",
          "c": "close_rate"
      })
)
# convert UNIX seconds → datetime index
df_fr["timestamp"] = pd.to_datetime(df_fr["timestamp"], unit="s")
df_fr = df_fr.set_index("timestamp")

df_fr.head()


,open_rate,high_rate,low_rate,close_rate
timestamp,,,,
2020-04-15,-0.012803,0.002925,-0.012803,0.002925
2020-04-16,-0.007017,0.006364,-0.046001,0.006364
2020-04-17,0.005926,0.005926,-0.006771,-0.006771
2020-04-18,-0.003132,-0.003132,-0.019642,-0.019642
2020-04-19,-0.019142,0.008514,-0.019142,0.008514


In [9]:
df_fr.to_csv('../output/bitcoin_funding_rate.csv')



In [ ]:
# Fetch Liquidation history from Coinalyze API
import os
import requests 
import pandas as pd

API_KEY = os.getenv("COINALYZE_API_KEY")
if not API_KEY:
    raise RuntimeError("COINALYZE_API_KEY not set in env")

SYMBOL   = "BTCUSDT_PERP.A"
INTERVAL = "daily"
START_TS = int(pd.to_datetime("2020-04-15").timestamp())
END_TS   = int(pd.to_datetime("2025-04-29").timestamp())
url = "https://api.coinalyze.net/v1/liquidation-history"
params = {
    "symbols":        SYMBOL,
    "interval":       INTERVAL,
    "from":           START_TS,
    "to":             END_TS,
    "api_key":        API_KEY
}
resp = requests.get(url, params=params)
resp.raise_for_status()  
data = resp.json()      
print(data[0]['history'])
# Picking BTC Liquidation history
hist = data[0]["history"]

df_liqh = (
    pd.DataFrame(hist)
      .rename(columns={
          "t": "timestamp",
          "l": "long_liq",
          "s": "short_liq",
      })
)
# convert UNIX seconds → datetime index
df_liqh["timestamp"] = pd.to_datetime(df_liqh["timestamp"], unit="s")
df_liqh = df_liqh.set_index("timestamp")
# Calculating ratio from liquidation
df_liqh['overall_liq'] = df_liqh['long_liq'] + df_liqh['short_liq']
df_liqh['long_ratio'] = df_liqh['long_liq'] / (df_liqh['long_liq'] + df_liqh['short_liq'])*100
df_liqh['short_ratio'] = df_liqh['short_liq'] / (df_liqh['long_liq'] + df_liqh['short_liq'])*100
df_liqh.drop(columns=['overall_liq'], inplace=True)

df_liqh.tail()

df_liqh.to_csv('../output/bitcoin_liquidation_history.csv')


[{'t': 1586908800, 'l': 2124.2729999999983, 's': 279.8729999999998}, {'t': 1586995200, 'l': 2839.6910000000075, 's': 5911.032000000031}, {'t': 1587081600, 'l': 754.5809999999994, 's': 393.4249999999998}, {'t': 1587168000, 'l': 226.80499999999992, 's': 1523.6069999999945}, {'t': 1587254400, 'l': 1320.551999999999, 's': 219.91899999999993}, {'t': 1587340800, 'l': 4359.979000000008, 's': 857.6659999999993}, {'t': 1587427200, 'l': 423.4409999999999, 's': 651.6619999999992}, {'t': 1587513600, 'l': 92.80400000000004, 's': 1899.09}, {'t': 1587600000, 'l': 1270.7269999999994, 's': 10147.271000000103}, {'t': 1587686400, 'l': 836.1169999999995, 's': 393.64100000000013}, {'t': 1587772800, 'l': 454.68099999999953, 's': 1232.6079999999995}, {'t': 1587859200, 'l': 1064.1209999999994, 's': 944.1279999999991}, {'t': 1587945600, 'l': 491.29899999999975, 's': 1021.6489999999989}, {'t': 1588032000, 'l': 282.95900000000006, 's': 114.23100000000005}, {'t': 1588118400, 'l': 3161.3060000000023, 's': 12660.32

In [12]:
# Merging 3

df_all = (
    df_oi
    .merge(df_fr, on='timestamp', how='outer')
    .merge(df_liqh, on='timestamp', how='outer')
)


df_all.to_csv('../output/bitcoin_derivatives_historical.csv')
df_all.tail(10)

,open_interest,high_oi,low_oi,close_oi,open_rate,high_rate,low_rate,close_rate,long_liq,short_liq,long_ratio,short_ratio
timestamp,,,,,,,,,,,,
2025-04-20,6.738089e+09,6.767681e+09,6.520097e+09,6.587516e+09,-0.000298,-0.000298,-0.002258,-0.002258,38.687,33.803,53.368741,46.631259
2025-04-21,6.767618e+09,7.538280e+09,6.692310e+09,7.179121e+09,0.005232,0.005403,0.001553,0.001553,45.189,91.100,33.156748,66.843252
2025-04-22,7.666597e+09,8.483385e+09,7.641351e+09,7.895156e+09,0.007749,0.007749,-0.007045,-0.007045,41.434,135.630,23.400578,76.599422
2025-04-23,7.914373e+09,8.146786e+09,7.608698e+09,7.641262e+09,-0.001389,0.004587,-0.001389,0.004587,75.588,111.377,40.428957,59.571043
2025-04-24,7.665713e+09,7.823968e+09,7.636459e+09,7.781190e+09,-0.002671,0.005210,-0.002671,-0.002165,34.719,74.541,31.776496,68.223504
2025-04-25,7.835888e+09,8.237823e+09,7.734183e+09,8.133263e+09,-0.001764,0.000170,-0.001764,0.000120,32.921,202.944,13.957560,86.042440
2025-04-26,8.131747e+09,8.136491e+09,7.911268e+09,7.931770e+09,-0.010085,0.002855,-0.010085,0.002855,14.837,38.810,27.656719,72.343281
2025-04-27,7.858064e+09,7.946920e+09,7.813984e+09,7.882132e+09,-0.001500,-0.000421,-0.001500,-0.000421,23.196,12.301,65.346367,34.653633
2025-04-28,7.989089e+09,8.091449e+09,7.745728e+09,7.776752e+09,0.000373,0.000373,-0.005821,-0.005821,47.434,55.260,46.189651,53.810349
